# Lab On You Own:  Player Efficiency Rating (PER)

## What is PER?  What does it do?


---

_Hat/tip Derek Topper_


So far we have studied offensive metrics for baseball.  One thing we have seen is that a lot of metrics are built as linear sums of positive and negative contributions.  We will dissect PER (Player Efficiency Rating) in this lab and observe how it works as a metric for player performance.

PER is a comprehensive metric that includes defensive statistics as well as offensive statistics.  While we have so far tried to segregate the two parts of play in baseball, we'll ignore that for now.

Basketball has a lot of moving parts and so the challenge is to figure out what the positive and negative contributions a player can make are and how to value them.

This notebook focuses on calculating Player Efficiency Rating given Raw NBA Player Data. PER looks like a complex, nasty equation but this notebook will break it down and show how it's not so complicated and how it makes an elegant attempt at performance measurenment.

For another in-depth look at PER, check out Justin Jacobs' blog about it:
https://squared2020.com/2017/09/01/breaking-down-player-efficiency-rating/


### TO DO

In the rest of the notebook, you will find ellipses in places where you need to fill in the required code (usually a formula).  At the end of the notebook, you'll find a set of questions to answer.

## 1. Setup and Data (Do not change)

In [1]:
%run ../../utils/notebook_setup.py

Adding datascience helper tools to path...
Setting up Matplotlib...
Matplotlib imported as mpl
Matplotlib.pyplot imported as plt


In [2]:
from datascience import Table, are
import numpy as np

### The Data

We'll be working the total stats of all players from the 2016-17 NBA season. The metrics contain each player's unique totals for statistics like Points, Assists, Rebounds, Blocks and Steals.

Here are the columns in the table below:
* Rk -- Rank
* Pos -- Position
* Age -- Age of Player at the start of February 1st of that season.
* Tm -- Team
* G -- Games
* GS -- Games Started
* MP -- Minutes Played
* FG -- Field Goals
* FGA -- Field Goal Attempts
* FG% -- Field Goal Percentage
* 3P -- 3-Point Field Goals
* 3PA -- 3-Point Field Goal Attempts
* 3P% -- 3-Point Field Goal Percentage
* 2P -- 2-Point Field Goals
* 2PA -- 2-point Field Goal Attempts
* 2P% -- 2-Point Field Goal Percentage
* eFG% -- Effective Field Goal Percentage
* FT -- Free Throws
* FTA -- Free Throw Attempts
* FT% -- Free Throw Percentage
* ORB -- Offensive Rebounds
* DRB -- Defensive Rebounds
* TRB -- Total Rebounds
* AST -- Assists
* STL -- Steals
* BLK -- Blocks
* TOV -- Turnovers
* PF -- Personal Fouls
* PTS -- Points


Our data has some players appearing more than once. That is because that player was traded or moved teams in the middle of the season. 

*For example:* Quincy Acy played 38 games total (TOT). Of those, 32 games were played for the Brooklyn Nets (BRK) and 6 were played for the Dallas Mavericks (DAL)

In [3]:
player_stats = Table().read_table('NBAPlayerStats2017.csv')
player_stats.show(5)

Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,Alex Abrines,SG,23,OKC,68,6,1055,134,341,0.393,94,247,0.381,40,94,0.426,0.531,44,49,0.898,18,68,86,40,37,8,33,114,406
2,Quincy Acy,PF,26,TOT,38,1,558,70,170,0.412,37,90,0.411,33,80,0.413,0.521,45,60,0.75,20,95,115,18,14,15,21,67,222
2,Quincy Acy,PF,26,DAL,6,0,48,5,17,0.294,1,7,0.143,4,10,0.4,0.324,2,3,0.667,2,6,8,0,0,0,2,9,13
2,Quincy Acy,PF,26,BRK,32,1,510,65,153,0.425,36,83,0.434,29,70,0.414,0.542,43,57,0.754,18,89,107,18,14,15,19,58,209
3,Steven Adams,C,23,OKC,80,80,2389,374,655,0.571,0,1,0,374,654,0.572,0.571,157,257,0.611,281,332,613,86,89,78,146,195,905


#### Remove TOT entries


In [4]:
player_stats = player_stats.where('Tm', are.not_equal_to('TOT'))

## 2. PER

John Hollinger was an NBA columnist for ESPN.com for eight years and is currently the Vice President of Basketball Operations for the Memphis Grizzlies. While at ESPN, he coined many advanced metrics in order to quantify player and team performance, such as Player Efficiency Rating (PER), Offensive Efficiency, Defensive Efficiency and Pace Factor.

PER is a rating of a player’s per-minute statistical performance that Hollinger developed to make player comparisons easier, and has become a widely used standard over the past decade. Hollinger has described PER as the sum of <a href http://www.espn.com/nba/columns/story?columnist=hollinger_john&id=2850240> "all a player's positive accomplishments, subtracts the negative accomplishments, and returns a per-minute rating of a player's performance."</a>

As PER attempts to be an all-encompassing number that looks at positive accomplishments, such as field goals, free throws, 3-pointers, assists, rebounds, blocks and steals, and negative results, such as missed shots, turnovers and fouls. 

The formula adds positive stats and subtracts negative ones through a statistical point value system. The rating for each player is then adjusted to a per-minute basis so that no player is negatively impcted by lack of playing time. It is also adjusted for pace of play. In the end, PER serves as one number that attempts to create an overall player score.


### PER Formula
$$
    aPER = uPER \times \frac{lgPace}{tmPace}, \quad PER = aPER \times \frac{15}{lgaPER} 
$$

Where:
* ''uPER'' stands for unadjusted PER
* ''aPER'' stands for pace-adjusted PER
* ''tm'', the prefix, indicating of team rather than of player;
* ''lg'', the prefix, indicating of league rather than of player;
* ''Pace'' is related to the style of play of a team. We'll also get there later.

The basic idea behind $uPER$ is the following:
\begin{align*}
uPER & = \frac{1}{min} \times \Bigg(\Bigg.\\
     & \quad\quad \text{Three Pointers Made} \\
     & \quad\quad + \text{Contributions from Assists} \\
     & \quad\quad + \text{Contributions from FGs} \\
     & \quad\quad + \text{Contributions from FTs} \\
     & \quad\quad - \text{Contributions from TOs} \\
     & \quad\quad - \text{Contributions from Missed FGs} \\
     & \quad\quad - \text{Contributions from Missed FTs} \\
     & \quad\quad + \text{Contributions from Def Rebounds} \\
     & \quad\quad + \text{Contributions from Off Rebounds} \\
     & \quad\quad + \text{Contributions from Steals} \\
     & \quad\quad + \text{Contributions from Blocks} \\
     & \quad\quad - \text{Contributions from Fouls} \\
     & \quad \Bigg.\Bigg)
\end{align*}

## 3. The Components of $uPER$
Let's do the computation for a specific player: Steph Curry.  We're not going to go in the order of the formula but rather start with some foundational quantities and then go from easiest to hardest computations.


In [5]:
curry = player_stats.row(109).asdict()
curry

OrderedDict([('Rk', 98),
             ('Player', 'Stephen Curry'),
             ('Pos', 'PG'),
             ('Age', 28),
             ('Tm', 'GSW'),
             ('G', 79),
             ('GS', 79),
             ('MP', 2638),
             ('FG', 675),
             ('FGA', 1443),
             ('FG%', 0.46799999999999997),
             ('3P', 324),
             ('3PA', 789),
             ('3P%', 0.41100000000000003),
             ('2P', 351),
             ('2PA', 654),
             ('2P%', 0.537),
             ('eFG%', 0.58),
             ('FT', 325),
             ('FTA', 362),
             ('FT%', 0.898),
             ('ORB', 61),
             ('DRB', 292),
             ('TRB', 353),
             ('AST', 524),
             ('STL', 142),
             ('BLK', 17),
             ('TOV', 239),
             ('PF', 183),
             ('PTS', 1999)])

### A. League Quantities

#### Value of Possession
$VOP$ is the value of a possession and is equal to
$$
    VOP = \frac{lgPTS}{lgFGA - lgORB + lgTO + 0.44 \times lgFTA}
$$
The denominator is an approximation to the number of possessions.

In [6]:
lgPTS = sum(player_stats.column('PTS'))
lgFGA = sum(player_stats.column('FGA'))
lgORB = sum(player_stats.column('ORB'))
lgTOV = sum(player_stats.column('TOV'))
lgFTA = sum(player_stats.column('FTA'))

vop = lgPTS/(lgFGA - lgORB + lgTOV + 0.44*lgFTA)
vop  # 1.0685413540268014

1.0685413540268014

### Defensive Rebound Percentage
Percentage of defensive rebounds grabbed is given by 
$$
    DRBP = \frac{lgTRB - lgORB}{lgTRB}
$$

In [7]:
lgTRB = sum(player_stats.column('TRB'))
lgORB = sum(player_stats.column('ORB'))

drbp = (lgTRB - lgORB)/lgTRB
drbp  # 0.7670440745100238

0.7670440745100238

### B. Contributions from Points

#### Three Pointers
Since three point shots are worth an extra point, we need to add in the extra point since it's not accounted for elsewhere.
$$
    \text{Three Pointers Made} = \mathit{3P}
$$

In [8]:
three_pt_contr = curry['3P']
three_pt_contr  # 324

324

#### Field Goals

PER values field goals in three ways:
1. You get 2 points for making any field goal
2. Some field goals are assisted.  We credit 2/3 of a point to the assister so we must deduct that from the 2 points in 1.
3. A further league correction is applied (no one seems to know what this does so we'll just have to accept it).

$$
\text{Contributions from FGs} = \left ( 2 - \frac23 \times \frac{tmAST}{tmFG} + K \times \frac{tmAST}{tmFG} \right ) \times FG
$$
where 
$$
    K =  \frac14 \times \frac{lgAST}{lgFG} \times \frac{lgFT}{lgFG}
$$


The usual way this calculation is presented is as,
$$
\text{Contributions from FGs} = \left ( 2 - \text{factor} \times \frac{tmAST}{tmFG} \right ) \times FG
$$
where discount the value of the FG from 2 to account for assists by using $\text{factor}$ multiplied by the team's assist rate.  The term $\text{factor} \times \frac{tmAST}{tmFG}$ is meant to capture the expected number of FGs which were assisted with $\text{factor}$ driving that expected value.  The largest component of $\text{factor}$ will be that $\frac23$ quantity.

In [26]:
# Team values
team = player_stats.where('Tm', "GSW")
tmAST = sum(team['AST'])
tmFG = sum(team['FG'])

# League values
lgAST = sum(player_stats['AST'])
lgFG = sum(player_stats['FG'])
lgFT = sum(player_stats['FT'])

# Factor
factor = (1/4) * (lgAST/lgFG) * (lgFT/lgFG)

# FGs
FG = curry['FG']

fg_contr = (2 - (2/3)*(tmAST/tmFG) + factor*(tmAST/tmFG))*FG
fg_contr  # 1064.1325273986868

1064.1325273986868

#### Free Throws

As with FGs, we need to discount FTs by the expected number of times they were assisted.  For a field goal, we discounted by 2/3. For free throws, we discount them by a lower amount: 1/6.

$$
    \text{Contributions from FTs} = \left ( 1 -  \frac{1}{6} \times \frac{tmAST}{tmFG} \right ) \times FT
$$

In [27]:
FT = curry['FT']
ft_contr = (1 - (1/6)*(tmAST/tmFG))*FT
ft_contr  # 286.79808418271045

286.79808418271045

### C. Rebounding and Assists

#### Assists

As we've seen, assists were determined to have value of $2/3$: an assist directly leads to a bucket but you shouldn't get full credit for the bucket. You get two-thirds of a point from the field goal.

$$ \text{Contributions from Assists} = \frac23 \times AST$$

In [28]:
asts_contr = 2/3 * curry['AST']
asts_contr  # 349.3333333333333

349.3333333333333

#### Defensive Rebounds

Since you are gaining a possession for your team, you should be rewarded for your rebounds but at the rate at which teams offensive rebound.

$$
    \text{Contributions from Def Rebounds} = VOP \times \left(1 - DRBP \right) \times \left(TRB - ORB \right)
$$

$VOP \times \left(1 - DRBP \right)$ represents the expected value of a possession for the opposing by their offensive rebounding.  By securing a defensive rebound, you prevent the opposing team from getting that value, hence why it's positively credited to you.  If teams secure all defensive rebounds, then $DRBP = 1$ and you get no credit because you did what everyone else does: secured a defensive rebound.  As $DRBP$ drops, the value of an individual defensive rebound goes up and you get credited more for each defensive rebound.



In [29]:
TRB = curry['TRB']
ORB = curry['ORB']

drb_contr = vop * (1 - drbp)*(TRB - ORB)
drb_contr  # 72.68552769507475

72.68552769507475

#### Offensive Rebounds

Similar to defensive rebounding, you are extending possession for your team and preventing the other team from gaining a possession.  You should be rewarded for the value of a possession at the rate at which teams defensive rebound.
$$
    \text{Contributions from Off Rebounds} = VOP \times DRBP \times ORB 
$$

$VOP \times DRBP$ represents the expected value of a possession for the opposing by their defensive rebounding.  By securing an offensive rebound, you prevent the opposing team from getting that possession value.  If teams secure nearly all defensive rebounds, then $DRBP \sim 1$ and you get a lot of credit for an offensive rebound because you prevented a possession.  As $DRBP$ drops, the value of an individual offensive rebounding drops and you get credited less for each offensive rebound.



In [30]:
orb_contr = vop * drbp * ORB
orb_contr  # 49.99671715248571

49.99671715248571

### D. Defense

#### Steals
Steals lead to a possession for the team so reward with $VOP$
$$
\text{Contributions from Steals} = VOP \times STL
$$

In [31]:
STL = curry['STL']
stl_contr = vop * STL
stl_contr  # 151.7328722718058

151.7328722718058

#### Blocks
Blocks are rewarded for gaining a possession at the rate at which they are rebounded (you shouldn't get rewarded for a block that is recovered by the other team).
$$
    \text{Contributions from Blocks} = VOP \times DRBP \times BLK 
$$

In [32]:
BLK = curry['BLK']
blk_contr = vop * drbp * BLK
blk_contr  # 13.933511337577983

13.933511337577983

### E. The Negatives

#### Turnovers
Turnovers prevent a chance at scoring so we need to dock the value of a possession from the player's rating.
$$
    \text{Contributions from TOs} = VOP \times TO
$$

In [33]:
TO = curry['TOV']
to_contr = vop * TO
to_contr  # 255.38138361240553

255.38138361240553

#### MIssed FGs
We need to dock the player for missed FGs that got rebounded by the defense.  A missed shot and no offensive rebound means a loss in the value of a possession.
$$
    \text{Contributions from Missed FGs} = VOP \times DRBP \times \left(FGA - FG \right) 
$$

In [34]:
FGA = curry['FGA']
missedfg_contr = vop * drbp * (FGA-FG)
missedfg_contr  # 629.4668651329348

629.4668651329348

#### Missed FTs

A missed free throw provides a value to the opposing team if it's not rebounded by the offense.  

We need to account for how missed FTs that didn't get rebounded by the offense led to a diminished value of the possession (not a full loss like a missed FG).  The arithmetic to account for this is given by,
$$
    \text{Contributions from Missed FTs} = VOP \times 0.44 \times \left(0.44 + 0.56 \times DRBP \right)
         \times \left(FTA - FT \right) 
$$

Like other aspects of PER, it's not immediately clear how this is supposed to work.  I'll let Justin Jacobs, who works for the Orlando Magic, try to explain it:


> Here, we calculate the number of missed free throws, $FTA - FT$. Next we have a deceitful term of $0.44 + 0.56 \times DRBP$. Recalling that $DRBP$ is the defensive rebound percentage, we can rewrite this as $0.44 \cdot ( 1 - DRBP) + DRBP$. The second term is the expected percentage of defensive rebounds on missed free throws that terminate possessions. We multiply by the extra 0.44 to ensure the expected terminated possession. The first term is the expected percentage of free throws that are offensively rebounded. There is an extra 0.44 term. The reason for this is due to the possession continuing for the same offensive team. In this case, if a field goal is attempted, the associated value is absorbed in another term. Hence, the free-throw only contributions are multiplied by a second 0.44 factor. Multiply this term by the league average points per possession and we obtain the expected number of points lost due to missed free throws.



In [35]:
FTA = curry['FTA']
missedft_contr = vop * 0.44 * (0.44 + 0.56 * drbp) * (FTA - FT)
missedft_contr  # 15.126471672013666

15.126471672013666

#### Fouls
Fouls lead to opposing points so you should be docked for giving up points.  You should only be docked for giving points above the expected value for those possessions.

\begin{align*}
    \text{Contributions from Fouls} & = \text{Total points from commmited fouls} - \text{Points expected on those possessions} \\
    & = 
    PF \times \frac{lgFT}{lgPF} - PF \times 0.44 \times \frac{lgFTA}{lgPF} \times VOP
\end{align*}

In [36]:
lgPF = sum(player_stats['PF'])

PF = curry['PF']
foul_contr = PF * (lgFT/lgPF) - PF * 0.44 * (lgFTA/lgPF) * vop
foul_contr  # 64.12348330192515

64.12348330192516

#### Curry's $\mathit{uPER}$

We put together all the contributions and we get $\mathit{uPER}$.

In [37]:
MP = curry['MP']
curry_uper = (
    three_pt_contr + asts_contr + fg_contr + ft_contr
    - to_contr - missedfg_contr - missedft_contr
    + drb_contr + orb_contr + stl_contr + blk_contr
    - foul_contr
) / MP
curry_uper  # 0.5111881613542061

0.5111881613542061

## 4. Computing PER

Recall the formula
$$
    \mathit{aPER} = \mathit{uPER} \times \frac{\mathit{lgPace}}{\mathit{tmPace}}, \quad \mathit{PER} = \mathit{aPER} \times \frac{15}{\mathit{lgaPER}} 
$$
where we now have $\mathit{uPER}$ for Steph Curry.

#### $\mathit{lguPER}$
We need to compute $\mathit{uPER}$ for every player.  We take the previous code and put it into a function to compute for every player.

In [38]:
def uPER(player, player_stats):
    # Team values
    team = player_stats.where('Tm', player['Tm'])
    tmAST = sum(team['AST'])
    tmFG = sum(team['FG'])

    # League values
    lgPTS = sum(player_stats['PTS'])
    lgFG = sum(player_stats['FG'])
    lgFGA = sum(player_stats['FGA'])
    lgAST = sum(player_stats['AST'])
    lgFT = sum(player_stats['FT'])
    lgFTA = sum(player_stats['FTA'])
    lgTRB = sum(player_stats['TRB'])
    lgORB = sum(player_stats['ORB'])
    lgTOV = sum(player_stats['TOV'])
    lgPF = sum(player_stats['PF'])

    # Values
    factor = (2 / 3) - (0.5 * (lgAST / lgFG)) / (2 * (lgFG / lgFT))
    vop = lgPTS / (lgFGA - lgORB + lgTOV + (.44 * lgFTA))
    drbp = (lgTRB - lgORB) / lgTRB

    # Stats
    MP = player['MP']
    FG3 = player['3P']
    FG = player['FG']
    FGA = player['FGA']
    AST = player['AST']
    FT = player['FT']
    FTA = player['FTA']
    TRB = player['TRB']
    ORB = player['ORB']
    STL = player['STL']
    TO = player['TOV']
    BLK = player['BLK']
    PF = player['PF']

    # Contributions
    three_pt_contr = FG3
    asts_contr = (2/3) * AST
    fg_contr = (2 - factor * tmAST / tmFG) * FG
    ft_contr = .5 * FT * (2 - tmAST / (3 * tmFG))
    to_contr = TO * vop 
    missedfg_contr = vop * drbp * (FGA - FG)
    missedft_contr = vop * .44 * (.44 + .56 * drbp) * (FTA - FT)
    drb_contr = vop * (1 - drbp) * (TRB - ORB)
    orb_contr = vop * drbp * ORB
    stl_contr = vop * STL
    blk_contr = vop * drbp * BLK
    foul_contr = PF * lgFT / lgPF - PF * .44 * lgFTA / lgPF * vop

    player_uper = (
        three_pt_contr + asts_contr + fg_contr + ft_contr
        - to_contr - missedfg_contr - missedft_contr
        + drb_contr + orb_contr + stl_contr + blk_contr
        - foul_contr
    ) / MP
    return player_uper

In [39]:
# verify
uPER(curry, player_stats), uPER(curry, player_stats) == curry_uper

(0.5111881613542061, True)

### A. Compute $\mathit{uPER$} for each player

In [40]:
player_upers = []
for player in player_stats.rows:
    player = player.asdict()
    # the function uPER takes two arguments, a dict like player and the full table player_stats
    player_uper = uPER(player, player_stats)
    player_upers.append(player_uper)
player_stats['uPER'] = player_upers

In [41]:
player_stats.show(20)

Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,uPER
1,Alex Abrines,SG,23,OKC,68,6,1055,134,341,0.393,94,247,0.381,40,94,0.426,0.531,44,49,0.898,18,68,86,40,37,8,33,114,406,0.205542
2,Quincy Acy,PF,26,DAL,6,0,48,5,17,0.294,1,7,0.143,4,10,0.4,0.324,2,3,0.667,2,6,8,0,0,0,2,9,13,-0.0274677
2,Quincy Acy,PF,26,BRK,32,1,510,65,153,0.425,36,83,0.434,29,70,0.414,0.542,43,57,0.754,18,89,107,18,14,15,19,58,209,0.275555
3,Steven Adams,C,23,OKC,80,80,2389,374,655,0.571,0,1,0,374,654,0.572,0.571,157,257,0.611,281,332,613,86,89,78,146,195,905,0.337159
4,Arron Afflalo,SG,31,SAC,61,45,1580,185,420,0.44,62,151,0.411,123,269,0.457,0.514,83,93,0.892,9,116,125,78,21,6,42,104,515,0.176202
5,Alexis Ajinca,C,28,NOP,39,15,584,89,178,0.5,0,4,0,89,174,0.511,0.5,29,40,0.725,46,131,177,12,20,22,31,77,207,0.262386
6,Cole Aldrich,C,28,MIN,62,0,531,45,86,0.523,0,0,nan,45,86,0.523,0.523,15,22,0.682,51,107,158,25,25,23,17,85,105,0.251501
7,LaMarcus Aldridge,PF,31,SAS,72,72,2335,500,1049,0.477,23,56,0.411,477,993,0.48,0.488,220,271,0.812,172,351,523,139,46,88,98,158,1243,0.364055
8,Lavoy Allen,PF,27,IND,61,5,871,77,168,0.458,0,1,0,77,167,0.461,0.458,23,33,0.697,105,114,219,57,18,24,29,78,177,0.232702
9,Tony Allen,SG,35,MEM,71,66,1914,274,595,0.461,15,54,0.278,259,541,0.479,0.473,80,130,0.615,166,225,391,98,115,29,100,178,643,0.25618


### B. Pace and Pace Factor

#### Team and League Pace
Team pace is stored in a separate file.

In [42]:
pace = Table().read_table('Pace.csv')
pace.sort('Pace', descending=True).show()

Team,Pace
BRK,101.3
PHO,100.3
HOU,100
GSW,99.8
LAL,98.5
PHI,98.5
DEN,98.3
NOP,98
OKC,97.8
ATL,97.4


In [43]:
def get_team_pace(pace, tm):
    return pace.where('Team', tm)['Pace'].item()

In [46]:
lgPace = get_team_pace(pace, 'League Average')
lgPace

96.4

#### Pace and Pace Factors for Each Player

We extract the player's team pace from the table `pace` and we compute the pace factor as
$$
    \text{Pace Factor} = \frac{lgPace}{tmPace}
$$

In [48]:
player_paces = []
player_pace_factors = []
for player in player_stats.rows:
    player = player.asdict()
    
    player_pace = get_team_pace(pace, player['Tm'])
    player_paces.append(player_pace)
    
    # compute the pace factor
    player_pace_factor = lgPace/player_pace
    player_pace_factors.append(player_pace_factor)
    
player_stats['Pace'] = player_paces
player_stats['Pace Factor'] = player_pace_factors

In [49]:
player_stats.show(5)

Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,uPER,Pace,Pace Factor
1,Alex Abrines,SG,23,OKC,68,6,1055,134,341,0.393,94,247,0.381,40,94,0.426,0.531,44,49,0.898,18,68,86,40,37,8,33,114,406,0.205542,97.8,0.985685
2,Quincy Acy,PF,26,DAL,6,0,48,5,17,0.294,1,7,0.143,4,10,0.4,0.324,2,3,0.667,2,6,8,0,0,0,2,9,13,-0.0274677,92.2,1.04555
2,Quincy Acy,PF,26,BRK,32,1,510,65,153,0.425,36,83,0.434,29,70,0.414,0.542,43,57,0.754,18,89,107,18,14,15,19,58,209,0.275555,101.3,0.951629
3,Steven Adams,C,23,OKC,80,80,2389,374,655,0.571,0,1,0,374,654,0.572,0.571,157,257,0.611,281,332,613,86,89,78,146,195,905,0.337159,97.8,0.985685
4,Arron Afflalo,SG,31,SAC,61,45,1580,185,420,0.44,62,151,0.411,123,269,0.457,0.514,83,93,0.892,9,116,125,78,21,6,42,104,515,0.176202,94.9,1.01581


### C. $\mathit{aPER}$

We adjust $uPER$ by the pace factor to get $aPER$.
$$
    \mathit{aPER} = \mathit{UPER} \times \frac{\mathit{lgPace}}{\mathit{tmPace}}
$$

In [53]:
player_stats['aPER'] = player_stats['uPER'] * (player_stats['Pace Factor'])

### D. Weighted Average to get $\mathit{lgaPER}$

We need to use a weighted average to get the $\mathit{lgaPER}$.  Why?  Suppose a player in one minute of action earned an extremely high $\mathit{aPER}$ while over a season, Russel Westbrook earned a lower (but still high) $\mathit{aPER}$.  Without weighting by minutes played, we'd naively treat these players evenly.

$$
    \mathit{lgaPER} = \sum_{\text{Players}} \frac{\mathit{MP}_{\text{Player $i$}}}{\text{Total MP by all players}} \times \mathit{aPER}_{\text{Player $i$}}
$$

In [54]:
weights = player_stats['MP'] / np.sum(player_stats['MP'])

# weighted average by minutes played
lg_aper = np.sum(player_stats['aPER'] * weights)
lg_aper

0.30084626863426245

In [55]:
player_stats.show(5)

Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,uPER,Pace,Pace Factor,aPER
1,Alex Abrines,SG,23,OKC,68,6,1055,134,341,0.393,94,247,0.381,40,94,0.426,0.531,44,49,0.898,18,68,86,40,37,8,33,114,406,0.205542,97.8,0.985685,0.2026
2,Quincy Acy,PF,26,DAL,6,0,48,5,17,0.294,1,7,0.143,4,10,0.4,0.324,2,3,0.667,2,6,8,0,0,0,2,9,13,-0.0274677,92.2,1.04555,-0.028719
2,Quincy Acy,PF,26,BRK,32,1,510,65,153,0.425,36,83,0.434,29,70,0.414,0.542,43,57,0.754,18,89,107,18,14,15,19,58,209,0.275555,101.3,0.951629,0.262226
3,Steven Adams,C,23,OKC,80,80,2389,374,655,0.571,0,1,0,374,654,0.572,0.571,157,257,0.611,281,332,613,86,89,78,146,195,905,0.337159,97.8,0.985685,0.332333
4,Arron Afflalo,SG,31,SAC,61,45,1580,185,420,0.44,62,151,0.411,123,269,0.457,0.514,83,93,0.892,9,116,125,78,21,6,42,104,515,0.176202,94.9,1.01581,0.178987


### E. Compute PER

In [56]:
player_stats['PER'] = player_stats['aPER'] * (15/lg_aper)

Display the top 20 in PER, restricting to players with over 1500 MP.

In [57]:
player_stats.where('MP', are.above_or_equal_to(1500)).\
    sort('PER', descending=True).\
    show(20)

Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,uPER,Pace,Pace Factor,aPER,PER
458,Russell Westbrook,PG,28,OKC,81,81,2802,824,1941,0.425,200,583,0.343,624,1358,0.459,0.476,710,840,0.845,137,727,864,840,132,31,438,190,2558,0.624007,97.8,0.985685,0.615074,30.6672
119,Kevin Durant,SF,28,GSW,62,62,2070,551,1026,0.537,117,312,0.375,434,714,0.608,0.594,336,384,0.875,39,474,513,300,66,99,138,117,1555,0.573128,99.8,0.965932,0.553603,27.6023
261,Kawhi Leonard,SF,25,SAS,74,74,2474,636,1312,0.485,147,387,0.38,489,925,0.529,0.541,469,533,0.88,80,350,430,260,133,55,154,122,1888,0.539479,94.2,1.02335,0.552079,27.5263
100,Anthony Davis,C,23,NOP,75,75,2708,770,1526,0.505,40,134,0.299,730,1392,0.524,0.518,519,647,0.802,172,712,884,157,94,167,181,168,2099,0.559593,98,0.983673,0.550457,27.4454
172,James Harden,PG,27,HOU,81,81,2947,674,1533,0.44,262,756,0.347,412,777,0.53,0.525,746,881,0.847,95,564,659,907,121,38,464,215,2356,0.569261,100,0.964,0.548768,27.3612
220,LeBron James,SF,32,CLE,74,74,2794,736,1344,0.548,124,342,0.363,612,1002,0.611,0.594,358,531,0.674,97,542,639,646,92,44,303,134,1954,0.540118,96.2,1.00208,0.541241,26.9859
424,Isaiah Thomas,PG,27,BOS,76,76,2569,682,1473,0.463,245,646,0.379,437,827,0.528,0.546,590,649,0.909,43,162,205,448,70,13,210,167,2199,0.533925,96.8,0.995868,0.531719,26.5111
90,DeMarcus Cousins,C,26,SAC,55,55,1891,505,1118,0.452,95,267,0.356,410,851,0.482,0.494,423,549,0.77,115,468,583,266,75,74,207,203,1528,0.522823,94.9,1.01581,0.531087,26.4796
233,Nikola Jokic,C,21,DEN,73,59,2038,494,854,0.578,45,139,0.324,449,715,0.628,0.605,188,228,0.825,212,506,718,359,61,55,171,214,1221,0.539483,98.3,0.980671,0.529055,26.3783
351,Chris Paul,PG,31,LAC,61,61,1921,374,785,0.476,124,302,0.411,250,483,0.518,0.555,232,260,0.892,40,264,304,563,119,8,147,146,1104,0.523136,96.1,1.00312,0.524769,26.1647


## 5. Why we need to pace adjust

Let's consider a non-pace adjusted version of PER using just $\mathit{uPER}$.

In [58]:
# compute weighted avarage lguPER
lg_uper = np.sum(player_stats['uPER'] * weights)

# compute non-pace adjusted PER
player_stats['PER_nopace'] =  player_stats['uPER'] * 15 / lg_uper

# compare to PER
player_stats['PER_diff'] = player_stats['PER_nopace'] - player_stats['PER']
player_stats['PER_ratio'] = player_stats['PER_nopace'] / player_stats['PER']

It shouldn't surprise you that the players with the largest positive difference between PER and the non-pace adjusted PER also play on teams with high pace (ie. pace factors < 1).  Conversely, the players with the largest negative differences play on teams with low pace (ie. pace factors > 1).

Overall differences between the two versions are about 5%.  While not huge, this can definetly rearrange perspective on a player by properly considering their pace adjustment.

In [59]:
player_stats.where('MP', are.above_or_equal_to(1500)).\
    sort('PER_diff', descending=True).\
    show(10)

Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,uPER,Pace,Pace Factor,aPER,PER,PER_nopace,PER_diff,PER_ratio
271,Brook Lopez,C,28,BRK,75,75,2222,555,1172,0.474,134,387,0.346,421,785,0.536,0.531,295,364,0.81,121,282,403,176,38,124,184,192,1539,0.429183,101.3,0.951629,0.408423,20.3637,21.4014,1.03774,1.05096
172,James Harden,PG,27,HOU,81,81,2947,674,1533,0.44,262,756,0.347,412,777,0.53,0.525,746,881,0.847,95,564,659,907,121,38,464,215,2356,0.569261,100,0.964,0.548768,27.3612,28.3865,1.02531,1.03747
119,Kevin Durant,SF,28,GSW,62,62,2070,551,1026,0.537,117,312,0.375,434,714,0.608,0.594,336,384,0.875,39,474,513,300,66,99,138,117,1555,0.573128,99.8,0.965932,0.553603,27.6023,28.5793,0.977066,1.0354
98,Stephen Curry,PG,28,GSW,79,79,2638,675,1443,0.468,324,789,0.411,351,654,0.537,0.58,325,362,0.898,61,292,353,524,142,17,239,183,1999,0.511188,99.8,0.965932,0.493773,24.6192,25.4907,0.871471,1.0354
49,Eric Bledsoe,PG,27,PHO,66,66,2176,449,1034,0.434,104,310,0.335,345,724,0.477,0.485,388,458,0.847,52,268,320,418,91,31,224,164,1390,0.428196,100.3,0.961117,0.411547,20.5194,21.3522,0.832789,1.04059
71,Clint Capela,C,22,HOU,65,59,1551,362,563,0.643,0,0,nan,362,563,0.643,0.643,94,177,0.531,178,348,526,63,34,79,87,179,818,0.446986,100,0.964,0.430894,21.4841,22.2892,0.805073,1.03747
54,Trevor Booker,PF,29,BRK,71,43,1754,305,591,0.516,25,78,0.321,280,513,0.546,0.537,74,110,0.673,143,428,571,138,75,28,127,152,709,0.330148,101.3,0.951629,0.314178,15.6647,16.463,0.798276,1.05096
200,Rondae Hollis-Jefferson,SF,22,BRK,78,50,1761,235,542,0.434,15,67,0.224,220,475,0.463,0.447,190,253,0.751,96,356,452,154,82,44,116,177,675,0.288183,101.3,0.951629,0.274243,13.6736,14.3704,0.696807,1.05096
246,Sean Kilpatrick,SG,27,BRK,70,24,1754,305,735,0.415,105,308,0.341,200,427,0.468,0.486,204,242,0.843,22,258,280,157,45,6,135,118,919,0.275043,101.3,0.951629,0.261739,13.0501,13.7152,0.665035,1.05096
454,T.J. Warren,SF,23,PHO,66,59,2048,403,814,0.495,26,98,0.265,377,716,0.527,0.511,119,154,0.773,125,214,339,75,76,39,57,175,951,0.326203,100.3,0.961117,0.313519,15.6318,16.2663,0.634424,1.04059


In [60]:
player_stats.where('MP', are.above_or_equal_to(1500)).\
    sort('PER_diff', descending=False).\
    show(10)

Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,uPER,Pace,Pace Factor,aPER,PER,PER_nopace,PER_diff,PER_ratio
155,Rudy Gobert,C,24,UTA,81,81,2744,413,625,0.661,0,1,0,413,624,0.662,0.661,311,476,0.653,314,721,1035,97,49,214,147,246,1137,0.444079,91.6,1.0524,0.46735,23.3017,22.1442,-1.15751,0.950325
185,Gordon Hayward,SF,26,UTA,73,73,2516,545,1156,0.471,149,374,0.398,396,782,0.506,0.536,362,429,0.844,49,345,394,252,73,20,140,117,1601,0.421175,91.6,1.0524,0.443245,22.0999,21.0021,-1.09781,0.950325
87,Mike Conley,PG,29,MEM,69,68,2292,464,1009,0.46,171,419,0.408,293,590,0.497,0.545,316,368,0.859,31,210,241,433,92,19,156,126,1415,0.446251,92.3,1.04442,0.466074,23.2381,22.2526,-0.985587,0.957588
194,George Hill,PG,30,UTA,49,49,1544,289,606,0.477,94,233,0.403,195,373,0.523,0.554,157,196,0.801,23,144,167,204,50,11,85,114,829,0.367159,91.6,1.0524,0.386399,19.2656,18.3086,-0.957014,0.950325
145,Marc Gasol,C,32,MEM,74,74,2531,532,1160,0.459,104,268,0.388,428,892,0.48,0.503,278,332,0.837,60,404,464,338,67,99,166,171,1446,0.389541,92.3,1.04442,0.406844,20.285,19.4246,-0.860336,0.957588
371,Zach Randolph,PF,35,MEM,73,5,1786,433,964,0.449,21,94,0.223,412,870,0.474,0.46,141,193,0.731,182,416,598,122,38,10,99,136,1028,0.355678,92.3,1.04442,0.371477,18.5216,17.7361,-0.785547,0.957588
28,Harrison Barnes,PF,24,DAL,79,79,2803,599,1280,0.468,78,222,0.351,521,1058,0.492,0.498,242,281,0.861,93,301,394,116,65,15,102,128,1518,0.310555,92.2,1.04555,0.324702,16.1894,15.486,-0.703429,0.95655
97,Seth Curry,PG,26,DAL,70,42,2029,338,703,0.481,137,322,0.425,201,381,0.528,0.578,85,100,0.85,25,154,179,189,79,7,92,126,898,0.295955,92.2,1.04555,0.309437,15.4283,14.758,-0.670359,0.95655
229,Joe Johnson,SF,35,UTA,78,14,1843,273,626,0.436,106,258,0.411,167,368,0.454,0.521,63,77,0.818,33,210,243,144,36,18,69,93,715,0.242661,91.6,1.0524,0.255377,12.7329,12.1004,-0.632504,0.950325
261,Kawhi Leonard,SF,25,SAS,74,74,2474,636,1312,0.485,147,387,0.38,489,925,0.529,0.541,469,533,0.88,80,350,430,260,133,55,154,122,1888,0.539479,94.2,1.02335,0.552079,27.5263,26.9014,-0.62486,0.9773


## 6. Questions

1. In your own words, what do you see as the overall goal of PER?
2. Where does PER use a style of expected value modeling in its calculations?
3. PER was developed in an era where basic box score statistics were the only thing really available.  Where does that lead to issues with how PER values a player's contributions to the team?
4. Lay out any criticisms of PER you might have.  Feel free to research further via google, but present the criticism in your own words.  Here's a prompt to get you started: One can argue that the NBA is a league of do everything superstars surrounded by specialists/role players who need to fit into a cog, especially on defense.  How does PER fail to evaluate players outside of the box score-stuffing stars?
5. What does PER do well?  Don't say "nothing".

## ANSWERS

1. The goal of the PER is to highlight players based on their contributions on the field as a standarized metric. It subtracts the negative impacts from the positive and returns a rating measuring efficiency.
2. When we adjust the aPER based on the uPER.
3. You may not get adjusted values from a simple box score because those statistics are raw, and only reflect the players performance obectively.
4. PER fails to evaulate the things that don't show up on the box score such as defensive stops, deflections etc. Players who do non-box score things well, don't have a PER rating for those catgories.
5. PER does a fair job evaluating players using their raw statistics (the ones that show up on the box plot). It can pull out the more efficient players from the stat-stuffers. It can show that Curry can do more with less and Westbrook can only do the same amount with more. 